# Homework 7
## Due Date:  Tuesday, October 24th at 11:59 PM

## Charles

### Part 1
A binary search tree is a binary tree with the invariant that for any particular node the left child is smaller and the right child is larger. Create the class BinaryTree with the following specifications:

\_\_init\_\_(self): Constructor takes no additional arguments

insert(self, val): This method will insert val into the tree

remove(self, val): This will remove the value from the tree. Upon removal the left child will take the place of the node if it exists, otherwise the right child. The subtree from the left or right child will propagate up in the same manner

getValues(self. depth): Return a list of the entire row of nodes at the specified depth with None at the index if there is no value in the tree. The length of the list should therefore be 2^depth. 

In [134]:
class BinaryTree:
    def __init__(self):
        self.data = [None]
        
    def insert(self, val):
        # keep track of idx we're at traversing the tree
        idx = 0
        while self.data[idx] is not None:
            idx = idx * 2 + (1 if self.data[idx] > val else 2)
            if idx >= len(self.data):
                self.data = self.data + [None]*(idx + 1 - len(self.data))
        self.data[idx] = val
        
    def find(self, val):
        idx = 0
        while self.data[idx] is not None and self.data[idx] != val:
            idx = idx * 2 + (1 if self.data[idx] > val else 2)
            if len(self.data) <= idx:
                return -1
        return idx if self.data[idx] is not None else -1
        
    def remove(self, val):
        # get idx of where val is
        idx = self.find(val)
        if idx >= 0:
            self.levelUp(idx)
    
    def levelUp(self, idx):
        self.data[idx] = None
        leftChild = idx * 2 + 1
        rightChild = (idx + 1) * 2
        if len(self.data) > leftChild:
            if self.data[leftChild] is not None:
                self.data[idx] = self.data[leftChild]
                self.levelUp(leftChild)
            elif len(self.data) > rightChild and self.data[rightChild] is not None:
                self.data[idx] = self.data[rightChild]
                self.levelUp(rightChild)
                
    def getValues(self, level):
        values = []
        for x in range(2**level - 1, 2**(level + 1) - 1):
            if len(self.data) <= x:
                values.append(None)
            else:
                values.append(self.data[x])
        return values

In [136]:
b = BinaryTree()
b.insert(5)
b.insert(3)
b.insert(8)
b.insert(2)
b.insert(1)
b.insert(4)
print(b.getValues(0))
print(b.getValues(1))
print(b.getValues(2))
print(b.getValues(3))
b.remove(3)
print()
print(b.getValues(0))
print(b.getValues(1))
print(b.getValues(2))
print(b.getValues(3))

[5]
[3, 8]
[2, 4, None, None]
[1, None, None, None, None, None, None, None]

[5]
[2, 8]
[1, 4, None, None]
[None, None, None, None, None, None, None, None]


### Part 2

Now we will write an iterator class to traverse the tree. There are three different types of traversal: preorder, inorder, and postorder. The definitions are laid out [here](https://en.wikipedia.org/wiki/Tree_traversal#Depth-first_search)

Write an iterator class DFSTraversal with the following specifications:

\_\_init\_\_(self, tree, traversalType): Constructor takes a BinaryTree object and one of the enums from DFSTraversalTypes

changeTraversalType(self, traversalType): Change the traversal type

\_\_iter\_\_(self): This is the initialization of an iterator

\_\_next\_\_(self): This is called in the iterator for getting the next value

In [22]:
from enum import Enum

class DFSTraversalTypes(Enum):
    PREORDER = 1
    INORDER = 2
    POSTORDER = 3

In [119]:
class DFSTraversal:
    def __init__(self, tree, traversalType):
        self.tree = tree
        self.traversalType = traversalType
        self.visited = set()
        
    def changeTraversalType(self, traversalType):
        self.traversalType = traversalType
        
    def __iter__(self):
        # initialize idx to the proper starting position
        idx = 0
        data = self.tree.data
        if self.traversalType != DFSTraversalTypes.PREORDER:
            # get the leftmost node
            while len(data) > idx*2 + 1 and data[idx*2 + 1] is not None:
                idx = idx*2 + 1
        self.idx = idx
        self.visited = set()
        return self

    def __next__(self):
        idx = self.idx
        data = self.tree.data
        if idx < 0 or len(data) <= idx or data[idx] is None:
            raise StopIteration
        leftIdx = idx * 2 + 1
        rightIdx = (idx + 1) * 2
        parentIdx = (idx - 1)//2
        traversalType = self.traversalType
        if traversalType == DFSTraversalTypes.PREORDER:
            #preorder just returns node when you reach it
            retVal = data[idx]
            retIdx = idx
            self.visited.add(retIdx)
            # finding next node to go to
            # if there's a left child then go there
            if len(data) > leftIdx and data[leftIdx] is not None:
                self.idx = leftIdx
            # otherwise if there's a right child go there
            elif len(data) > rightIdx and data[rightIdx] is not None:
                self.idx = rightIdx
            # otherwise have to do some backtracking
            else:
                # find the parent who has a right child that's unvisited
                idx = parentIdx
                while idx >= 0 and not ((idx + 1)*2 < len(data) and data[(idx + 1)*2] is not None and (idx + 1)*2 not in self.visited):
                    idx = (idx - 1)//2
                if idx < 0:
                    self.idx = len(data)
                else:
                    self.idx = (idx + 1)*2
            return retVal
        elif traversalType == DFSTraversalTypes.INORDER:
            retVal = data[idx]
            retIdx = idx
            # if we have a right child node then we need to traverse that side
            if len(data) > rightIdx and data[rightIdx] is not None:
                idx = rightIdx
                while idx * 2 + 1 < len(data) and data[idx*2 + 1] is not None:
                    idx = idx*2 + 1
            else:
                # otherwise we go to the next unvisited parent
                idx = parentIdx
                while idx >= 0 and idx in self.visited:
                    idx = (idx - 1)//2
            self.idx = idx
            self.visited.add(retIdx)
            return retVal
        elif traversalType == DFSTraversalTypes.POSTORDER:
            retVal = data[idx]
            retIdx = idx
            self.visited.add(retIdx)
            # go to next unvisited parent
            idx = parentIdx
            rightChild = (idx + 1)*2
            # if there is an unvisited right child, traverse down children of right child, otherwise parent is the next idx
            if rightChild < len(data) and data[rightChild] is not None and rightChild not in self.visited:
                idx = rightChild
                leftChild = idx*2 + 1
                rightChild = (idx + 1)*2
                while (leftChild < len(data) and data[leftChild] is not None) or (rightChild < len(data) and data[rightChild] is not None):
                    idx = leftChild if data[leftChild] is not None else rightChild
                    leftChild = idx*2 + 1
                    rightChild = (idx + 1)*2
                self.idx = idx
            self.idx = idx
            return retVal

### Part 3
Put both classes in a file titled TreeTraversal.py

In [123]:
def testFunction(inputArr):
    b = BinaryTree()
    for val in inputArr:
        b.insert(val)
    traversal = DFSTraversal(b, DFSTraversalTypes.PREORDER)
    traversalTrace = {}
    for traversalType in DFSTraversalTypes:
        traversal.changeTraversalType(traversalType)
        traversalTrace[traversalType.name] = []
        for val in traversal:
            traversalTrace[traversalType.name].append(val)
    return traversalTrace

In [126]:
#wikipedia example
testFunction(['F', 'B', 'A', 'D', 'C', 'E', 'G', 'I', 'H'])

{'INORDER': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
 'POSTORDER': ['A', 'C', 'E', 'D', 'B', 'H', 'I', 'G', 'F'],
 'PREORDER': ['F', 'B', 'A', 'D', 'C', 'E', 'G', 'I', 'H']}